# Automation of MODE Wrapper

This notebook downnloads all data required for METplus's MODE wrapper to verify the performance of forecast model data to that of observation. The forecast data in this case is HRRR data which is obtained from https://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/hrrr_download.cgi. The observation data is lowest level reflectivity from MRMS https://mesonet.agron.iastate.edu/GIS/rasters.php?rid=4.

In [1]:
#Import Modules
import requests
import xarray as xr
from datetime import datetime, timedelta
import cfgrib
import netCDF4 as nc
import os
import glob
import multiprocessing

In [2]:
def MODE_Wrapper(data_file_path):
    """this is a function built to run the MODE wrapper with multiprocessing.
        MODE is an object based verfication method developed by NCAR and DTC.
        
        data_file_path = Path to the file including the file as well.
        
        Will return 4 files that the MODE wrapper creates.
        More information can be found at https://dtcenter.org/met-online-tutorial-metv8-0/mode/configure"""
    
    
    os.system(f'python /gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/ush/run_metplus.py -c {data_file_path}')

## Observation Data Download

In [3]:
#Create a datetime object to grab data for.
datetime_object = datetime(2020, 6, 30, 0)

#obtain the file for the MRMS lowest level reflectivity from the Iowa State website.
#set the data and pathfile as well as create the directory.
url = f'https://mesonet.agron.iastate.edu/cgi-bin/request/raster2netcdf.py?dstr={datetime_object.strftime("%Y%m%d%H00")}&prod=mrms_lcref'
drt = f'/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus_input/MODE_data/{url[-28:-16]}'
os.mkdir(drt)

#Get data from the website and write it to a file.
response = requests.get(url)
filepath = f'{drt}/mrms_lcref{url[-28:-16]}.nc4'
open(filepath, 'wb').write(response.content)

1237210

In [4]:
#find all the configuration files for each forecast hour that we will use.
config_file = glob.glob(f'/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/*.conf')
config_file

['/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FCST_RADAR_OBS_f30.conf',
 '/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FCST_RADAR_OBS_f15.conf',
 '/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FCST_RADAR_OBS_f09.conf',
 '/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FCST_RADAR_OBS_f06.conf',
 '/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FCST_RADAR_OBS_f36.conf',
 '/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FCST_RADAR_OBS_f12.conf',
 '/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FCST_RADAR_OBS_f18.conf',
 '/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/MODE_HRRR_FC

## Forecast Data Download & Editing of Configuration File

In [ ]:
#array of all the forecast hours to get data for.
forecast_hours  = [3, 6, 9, 12, 15, 18, 24, 30, 36]

#iterate through the forecast hours 
for hours in forecast_hours:
    
    #calculate the model run that will line up with the correct forecast hour.
    new_hour = datetime_object - timedelta(hours=hours)
    
    #Get data from the websit and write it to a file.
    url = f'https://pando-rgw01.chpc.utah.edu/hrrr/sfc/{new_hour.strftime("%Y%m%d")}/hrrr.t{new_hour.strftime("%H")}z.wrfsfcf{str(hours).zfill(2)}.grib2'
    response = requests.get(url)
    open(f'{drt}/hrrr_t{new_hour.strftime("%Y%m%dT%H00")}z_wrfsfcf{str(hours).zfill(2)}.grib2', 'wb').write(response.content)
    
    #the configuration file that will be changed.
    change_file = glob.glob(f'/gpfs/fs1/home/ac.jcorner/NIU/Masters_Thesis/metplus/metplus/parm/metplus_config/MODEconf/*{str(hours).zfill(2)}.conf')
    
    #open the file and read each line.
    w = open(fr'{change_file[0]}', 'r')
    content = w.readlines()
    
    #search input datafile lines and and replace them with the new files.
    for_search_text = content[99][-50:-1]
    obs_search_text = content[101][-40:-1]
    
    for_replace_text = f'{drt[-12:-1]}0/hrrr_t{new_hour.strftime("%Y%m%dT%H00")}z_wrfsfcf{str(hours).zfill(2)}.grib2'
    obs_replace_text = f'{drt[-12:-1]}0/mrms_lcref{drt[-12:-1]}0.nc4'
    
    #read in the file for the purpose of editing it. 
    with open(fr'{change_file[0]}','r') as f:

        #Reading the content of the file
        data = f.read()

        # Searching and  replacing the text
        data = data.replace(for_search_text, for_replace_text)
        data = data.replace(obs_search_text, obs_replace_text)


        #edit the old file and add the new datafiles instead.
        with open(fr'{change_file[0]}', 'w') as file:

            # Writing the replaced data.
            file.write(data)
            file.close()


In [ ]:
#use multiprocessing to run all the different forecast hours at the same time. 
for paths in range(len(config_file)):
    d = multiprocessing.Process(target=MODE_Wrapper, args=[config_file[paths]])
    d.start()